In [2]:
### GET THE FILE PATH FOR THE WORKING FOLDER
# Import the librart
import os
# method to return the file path
cwd = os.getcwd()
# print result
print(cwd)

C:\Users\GavinAJohnson\Python Projects\UCD Cert in Data Analytics


In [ ]:
### Data set import of the GLEIF Daily files from ThE UnaVista.lseg.com website.

# https://stackabuse.com/parsing-xml-with-beautifulsoup-in-python/
#Import the Modules required.
from bs4 import BeautifulSoup
import pandas as pd

### LOAD DATA CONFIGURATION
### File Path
FILE_PATH = "/Users/GavinAJohnson/Python Projects/UCD Cert in Data Analytics/Data/"

### FILENAME
# LSEG Daily File from may 15, 2023
GLEIF_data_file = FILE_PATH + "lei2-full-files LEI_Full_File_20230515.xml"
# Test QCB File as much smaller.
#GLEIF_data_file = FILE_PATH + "20230424-254900PMALKJRL1YGQ18-file.xml"

#with open('20230424-254900PMALKJRL1YGQ18-file.xml', 'r', encoding="utf-8") as f:  
with open(GLEIF_data_file, 'r', encoding="utf-8") as f:
    xml_data = f.read()

# Initializing soup variable
soup = BeautifulSoup(xml_data, 'xml') 

# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object

# Creating column for table
df = pd.DataFrame(columns=['lei', 'legal_name', 'city', 'country', 'ra_code', 'ra_id', 'elf_code', 'other_legal_form', 'registration_date', 'reg_status', 'last_upadate_date', 'renewal_date'])
print(df.shape)

# Iterating through item tag and extracting elements
all_items = soup.find_all('LEIRecord')
items_length = len(all_items)

print(items_length)
# None error https://stackoverflow.com/questions/51664292/python-attributeerror-nonetype-object-has-no-attribute-text


for index, item in enumerate(all_items):
    lei = item.find('LEI').text
    legal_name = item.find('LegalName').text
    city = item.find('LegalAddress').find('City').text
    country = item.find('LegalAddress').find('Country').text
    ra_code = item.find('RegistrationAuthorityID').text
    ra_id = item.find('RegistrationAuthorityEntityID')
    elf_code = item.find('EntityLegalFormCode').text
    # New code added due to the performance of the API call code below.
    # Check the ELF code is 8888 otherwise OtherLegalForm won't exist.
    if elf_code == '8888':
        other_legal_form = item.find('OtherLegalForm').text
    else:
        other_legal_form = ""
    registration_date = item.find('InitialRegistrationDate').text
    reg_status = item.find('RegistrationStatus').text
    last_update_date = item.find('LastUpdateDate').text
    renewal_date = item.find('NextRenewalDate').text

    # Adding extracted elements to rows in table
    row = {
            'lei': lei,
            'legal_name': legal_name,
            'city': city,
            'country': country,
            'ra_code': ra_code,
            'ra_id': ra_id,
            'elf_code': elf_code,
            'other_legal_form': other_legal_form,
            'registration_date': registration_date,
            'reg_status': reg_status,
            'last_update_date': last_update_date,
            'renewal_date': renewal_date
        }
    print(index) # Use to show progress.

    # Create the new row as its own dataframe
    #https://stackoverflow.com/questions/70837397/good-alternative-to-pandas-append-method-now-that-it-is-being-deprecated
    #https://stackoverflow.com/questions/17839973/constructing-pandas-dataframe-from-values-in-variables-gives-valueerror-if-usi
        
    
    # Create the new row to add to the DataFrame, set lei to be the index.
    df_new_row = pd.DataFrame(row, index=[lei])
    # Add the new row to the DataFrame.
    df = pd.concat([df, df_new_row])


     
#Save the New DataFrame to CSV.
df.to_csv('lei_extract.csv')

# Print the Info of the dataframe. 
print(df.info())



In [3]:
### Process the date columns in the DataFrames for use in the charts. 
### Import Modules
import pandas as pd
import datetime
from pathlib import Path 
import dateutil.parser

### File Path
FILE_PATH = "/Users/GavinAJohnson/Python Projects/UCD Cert in Data Analytics/"

### FILENAME
### LEI Data
# File Created from the XML import.
lei_extract_file_path = FILE_PATH + "lei_extract_new_20230515.csv"

# Name of new file to be created. 
lei_extract_with_dates = FILE_PATH + "lei_extract_dates_new_20230515.csv"


# Date Change Function
def date_substring(str_date):
    """Function take the substring of the 1st 10 characters of the date"""
    datetime_str ="" 
    if len(str(str_date)) >= 10:    
        datetime_str = str(str_date)
        datetime_str = datetime_str[0:10]
    else:
        datetime_str
    
    return datetime_str

# Test the date_substring() Function
#print(date_substring("2015-12-18T00:00:00Z")) ### '%Y-%m-%d %H:%M:%S.%f'

# Read the CSV into the DataFrame
df_lei_list = pd.read_csv(lei_extract_file_path)

# Check for nulls
# df_lei_list.isna()

# use the function to Trim the strings and convert to dates.
df_lei_list['renewal_date'] = pd.to_datetime(df_lei_list['renewal_date']).apply(date_substring)
df_lei_list['registration_date'] = pd.to_datetime(df_lei_list['registration_date']).apply(date_substring)

# Test the Result
#print(df_lei_list['renewal_date'])
#print(df_lei_list['registration_date'])

# Show the Data Frame Head (first 5 Rows) and Info
#df_lei_list.head()
#df_lei_list.info()

# Take the date strings in the convert them to dates, set them as indexes and extract month and year to new columns. 
# https://towardsdatascience.com/formating-and-visualizing-time-series-data-ba0b2548f27b

#renewal_date
df_lei_list['renewal_date'] = pd.to_datetime(df_lei_list['renewal_date'])
df_lei_list = df_lei_list.set_index("renewal_date")
# Extarct the month and year to new columns
df_lei_list["ren_year"] = df_lei_list.index.year
df_lei_list["ren_month"] = df_lei_list.index.month

#Reset the Index
df_lei_list.reset_index(inplace=True)

# registration_date
df_lei_list['registration_date'] = pd.to_datetime(df_lei_list['registration_date'])
df_lei_list = df_lei_list.set_index("registration_date")
# Extarct the month and year to new columns
df_lei_list["reg_year"] = df_lei_list.index.year
df_lei_list["reg_month"] = df_lei_list.index.month

#Reset the Index
df_lei_list.reset_index(inplace=True)

# last_update_date
df_lei_list['last_update_date'] = pd.to_datetime(df_lei_list['last_update_date'])
df_lei_list = df_lei_list.set_index("last_update_date")
# Extarct the month and year to new columns
df_lei_list["update_year"] = df_lei_list.index.year
df_lei_list["update_month"] = df_lei_list.index.month

#Reset the Index
df_lei_list.reset_index(inplace=True)
df_lei_list = df_lei_list.set_index("lei")

### Save the new DataFrame witht the new date columns to CSV 
df_lei_list.to_csv('lei_extract_dates_20230515.csv')  

### Test Prints on the new DataFrame created
df_lei_list.info()    
print(df_lei_list.head())
df_lei_list.shape

<class 'pandas.core.frame.DataFrame'>
Index: 215588 entries, 213800Q7JGGZ17BFJK36 to 213800J62FPNBUATEZ23
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   last_update_date   215588 non-null  datetime64[ns, UTC]
 1   registration_date  215588 non-null  datetime64[ns]     
 2   renewal_date       215588 non-null  datetime64[ns]     
 3   Unnamed: 0         215588 non-null  object             
 4   legal_name         215588 non-null  object             
 5   city               215588 non-null  object             
 6   country            215578 non-null  object             
 7   ra_code            215588 non-null  object             
 8   ra_id              140602 non-null  object             
 9   elf_code           215588 non-null  object             
 10  other_legal_form   111349 non-null  object             
 11  reg_status         215588 non-null  object             
 12  la

(215588, 19)

In [ ]:
### CREATE A DATAFRAME OF ALL LEIs WITH ELF_CODE 8888

# Import the Pandas Module.
import pandas as pd

### LOAD DATA CONFIGURATION.
### File Path
FILE_PATH = "/Users/GavinAJohnson/Python Projects/UCD Cert in Data Analytics/Data/"

# FILENAME
# LEI Data - with dates split into year and month colums
lei_extract_file_path = FILE_PATH + "lei_extract_dates_20230515.csv"

# LOAD DATA INTO THE DATAFRAMES
df_lei_data = pd.DataFrame(pd.read_csv(lei_extract_file_path,","))

### Extract the files with ELF code 8888
df_elf_code_8888 = df_lei_data[df_lei_data['elf_code'].isin(['8888'])]
# Set the Index of the DataFrame.
df_elf_code_8888 = df_elf_code_8888.set_index("lei")

# TestPrint
print(df_elf_code_8888.head())

# Save DataFrame to new CSV.
df_elf_code_8888.to_csv('lei_elf_code_8888_20230515.csv')


In [4]:
### Function to get the Other Legal Form from the GLEIF API
#Sample LEI lei = 213800KPY517T1EP5K86
import requests
def get_other_legal_form(lei):
    """ Function to access the Other Legal Form in the JSON result returned from the GLEIF API"""
    # Send te request to the API
    api_request = requests.get('https://api.gleif.org/api/v1/lei-records/' + lei, headers={'Accept': 'application/json'})
    # Response Returned
    response_dict = api_request.json()
    # Extract the Other Legal Form element.
    try:
        # https://stackoverflow.com/questions/25833613/safe-method-to-get-value-of-nested-dictionary
        other_legal_form = response_dict.get("data",{}).get("attributes",{}).get("entity",{}).get("legalForm",{}).get("other",{})
    except:
        other_legal_form = ""
    
    return other_legal_form

# Test the Function
print(get_other_legal_form('213800KPY517T1EP5K86'))

TRUST


In [ ]:
### USE THE GLEIF API to Return the the OtherLegalForm for the LEIs with ELF_CODE 8888 

### Although this code works, is was processing about a record every second so the 110,000 records would take about 30 hours, 
### in addition it would stall after about 5000 queries of the API.
### as a result the original update code was updated to process the OtherLegal Forms.

# Import the Libraries.
import requests
import json
import csv         # For the import of the CSV data files

# File Containing the LEIs with ELF CODE 8888
lei_data_file_path = "lei_elf_code_8888_20230515.csv"

# New File name to Save the Dataset.
# CSV Format LEI, OtherLegalForm
file_name = "elf_code_8888.csv"

# GLEIF Documentation
# https://documenter.getpostman.com/view/7679680/SVYrrxuU#5fc83577-f5ae-41fa-aa4e-2848554958b4
# Example API Call - https://api.gleif.org/api/v1/lei-records/529900W18LQJJN6SJ336
def get_other_legal_form(lei):
    """ Function to access the Other Legal Form in the JSON result returned from the GLEIF API"""
    # Send te request to the API
    api_request = requests.get('https://api.gleif.org/api/v1/lei-records/' + lei, headers={'Accept': 'application/json'})
    # Response Returned
    response_dict = api_request.json()
    
    # The Other Legal Form does now always exist so try / except to ensure the code runs. 
    # Extract the Other Legal Form element.
    try:
        # https://stackoverflow.com/questions/25833613/safe-method-to-get-value-of-nested-dictionary
        other_legal_form = response_dict.get("data",{}).get("attributes",{}).get("entity",{}).get("legalForm",{}).get("other",{})
    except:
        other_legal_form = ""
    
    return other_legal_form

### Function to write the line to CSV.
def write_to_file(file_name, file_csv_line):
    """ Function to write new line to the CSV"""
    #Open the file in append 'a' mode - utf-8 is required as there are extended characters.
    f = open(file_name, "a", encoding="utf-8")
    f.write(file_csv_line)
    
    return

# Test the Functions
# print(get_other_legal_form('213800R1WEDMWDNIS110'))
# file_csv_line = '213800R1WEDMWDNIS110'+ "," + get_other_legal_form('213800R1WEDMWDNIS110') + "\n"
# write_to_file(file_name, file_csv_line)
# print(file_csv_line)

### Open the file containt the LEIs with ELF CODE 8888
try:
    #lei_data_file_path = FILE_PATH + "lei-records.csv"
    file = open(lei_data_file_path, "r", encoding='utf-8')
    # Read the file into a list.
    lei_data = list(csv.reader(file, delimiter=","))
    # close the file
    file.close()

    lei_data.pop(0)              ### Remove the title row
    total_leis = len(lei_data)   ### Count the number of items in the list of LEIs
    print(len(lei_data))


except Exception as e:
    print("There has been an error Loading the LEI Data:" + str(e))

# Confirm the type of the lei_data list    
# print(type(lei_data))

lei_list = []                     # Initiate the list
for lei in lei_data:              # for all the lines in the list
    lei_list.append(lei[0])       # add the first item (the LEI number)


### The For loop below would time out after about 5000 queries so was replaced with the one below.
# for lei in lei_list:
#     file_csv_line = lei + "," + get_other_legal_form(lei) + "\n"      # create the file line
#     write_to_file(file_name, file_csv_line)                           # write it the the file
#     print(lei)                                                        # print to show progress

# Last LEI to be processed and written to the CSV before timeout.
last_lei = "21380013E3NKIVRZJS54"
print(lei_list.index(last_lei))               # Find the index of the last LEI processed so when timed out could restart at next LEI
int_continue = lei_list.index(last_lei) + 1   # add 1 to start at the next one

print(int_continue)                           # show the position

# Set i for the while loop to the next item to be processed.
i = int_continue

# Start the loop:
while i < len(lei_list):
    lei = lei_list[i]                         # Starts at the value passed in.
    # Create the line to be written to the CSV '\n' for new line
    file_csv_line += lei + "," + get_other_legal_form(lei) + "\n"
    
    write_to_file(file_name, file_csv_line)   # Write the line to the file.
    print(lei)                                # Print the LEI to show progress.
    i = i + 1                                 # Move to next record


